把inputs data timeframe 從全部時間段變成一個月一個時間段去預測(太久遠的資料意義可能不大)

In [109]:
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import copy
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, accuracy_score, recall_score, precision_score, f1_score

from pathlib import Path
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from datetime import date
from pathlib import Path
from keras.layers import LSTM, Bidirectional, Dropout, Dense, TimeDistributed, Conv1D, AveragePooling1D, BatchNormalization, Flatten, MaxPooling1D

In [132]:
#讀資料

data_loc = "D:\\論文相關\\data\\"
text_loc = data_loc+"text_loc\\"
emotion_loc = data_loc+"emotion_data\\"
#dict_emo_list = pd.read_pickle(emotion_loc + "dict_emotion_list.pkl")
#dict_classified_emo_list = pd.read_pickle(emotion_loc + "dict_classified_emotion_list.pkl")
#dict_classified_emo_list_with_neu = copy.deepcopy(dict_classified_emo_list)
#classified_emo_list = pd.read_pickle(emotion_loc + "classified_emotion_list.pkl")

#這是分字數後的dict emotion list
dict_emo_list = pd.read_pickle("D:\\論文相關\\data\\emotion_data\\dict_emo_list.pickle") #最後一個是text limit = 0(全部資料集的emo)

#這是分字數後的self labeled +dict labeled emotion list
classified_emo_list = pd.read_pickle("D:\\論文相關\data\\classifier emotions\\classified_emotion_list_len_0.pkl")

for idx in range(len(dict_emo_list)):
    dict_emo_list[idx] = pd.DataFrame(dict_emo_list[idx]).T
    columns_to_drop = dict_emo_list[idx].columns
    dict_emo_list[idx]["pos"] = dict_emo_list[idx]["年報正面"]+  dict_emo_list[idx]["社媒正面"]
    dict_emo_list[idx]["neg"] = dict_emo_list[idx]["年報負面"]+  dict_emo_list[idx]["社媒負面"]
    
    dict_emo_list[idx]["pos_ratio"] = dict_emo_list[idx]["pos"] /(dict_emo_list[idx]["neg"]+dict_emo_list[idx]["pos"])
    dict_emo_list[idx]["neg_ratio"] = dict_emo_list[idx]["neg"] /(dict_emo_list[idx]["neg"]+dict_emo_list[idx]["pos"])
    dict_emo_list[idx] = dict_emo_list[idx].loc["2020-04-20":,:]
    dict_emo_list[idx] = dict_emo_list[idx].drop(columns_to_drop, axis = 1)

#把dict labeled的情緒加到self labeled上面
dict_classified_emo_list = []
for idx in range(len(classified_emo_list)):
    cur_classified = classified_emo_list[idx]
    cur_dict = dict_emo_list[idx]
    columns_to_drop = cur_classified.columns
    cur_classified["pos"] = cur_dict["pos"] + cur_classified["classified_pos"]
    cur_classified["neg"] = cur_dict["neg"] + cur_classified["classified_neg"]

    cur_classified["pos_ratio"] = cur_classified["pos"]/(cur_classified["pos"]+ cur_classified["neg"])
    cur_classified["neg_ratio"] = cur_classified["neg"]/(cur_classified["pos"]+ cur_classified["neg"])
    cur_classified = cur_classified.drop(columns_to_drop, axis = 1)
    dict_classified_emo_list.append(cur_classified)
    
#dict_emo_list = pd.read_pickle(emotion_loc + "dict_emotion_list.pkl") #因為上述行為會覆蓋到原本list 重讀一次
#CFSD_loc = text_loc+"emotion_CFSD\\"
#new_emo_loc = text_loc + "new_emotion_pnt\\"
#new_emotion_with_middel_dir = Path(new_emo_loc +"with middele")
#new_emotion_without_middel_dir = Path(new_emo_loc +"without middle")
#classified_loc = data_loc +'classifier emotions\\'
#all_classified_emo = classified_loc+"classed_daily_emotion.pkl"

In [133]:
dict_emo_list[4].loc["2020-04-20",:]

pos          40.000
neg          24.000
pos_ratio     0.625
neg_ratio     0.375
Name: 2020-04-20, dtype: float64

In [134]:
#跑F1 或是 acc的表格出來
def retrain_results(trained_results, metric = f1_score, if_emo = True):
    DF = {}
    day_period = 1
    for result  in trained_results:
        text_len = 3
        DF[f"moving_avg_days "+str(day_period)] ={}
        for cur_result in result:
            predict_up_down = get_up_down_pd(cur_result[0])
            true_up_down = get_up_down_pd(cur_result[1])
            #print(f"moving_avg of {day_period} days and sentence_length limit of minimum {text_len} words")
            #print("acc:", accuracy_score(true_up_down["up_down"], predict_up_down["up_down"]))
            #print("recall:",recall_score(true_up_down["up_down"], predict_up_down["up_down"]))
            #print("precision:",precision_score(true_up_down["up_down"], predict_up_down["up_down"]))
            #print("F1:",f1_score(true_up_down["up_down"], predict_up_down["up_down"]))
            #print("\n")
            DF[f"moving_avg_days "+str(day_period)]["text_len "+ str(text_len)] = metric(true_up_down["up_down"], predict_up_down["up_down"])
            text_len+=1
        day_period+=1
    DF = pd.DataFrame(DF)
    if not if_emo:
        DF.index = ["Technical_indicators_only"]
    return DF
    

In [135]:
#標註漲跌
def get_up_down_pd(results, n =1):
    cur_pd = pd.DataFrame(results)
    cur_pd["return"] = (cur_pd[0]- cur_pd[0].shift(n))/cur_pd[0]
    cur_pd = cur_pd.fillna(0)
    cur_pd["up_down"] = cur_pd["return"].map(get_up_down)
    
    return cur_pd
def get_up_down(data, bounary = 0):
    if data>bounary:
        return 1
    else:
        return -1


# 訓練資料前處理

In [6]:
#台灣指數
taiwan_idx = pd.read_csv('TWII.csv')
idx_simed = pd.read_csv("simulated_tw_idx.csv")
idx_simed = idx_simed.set_index("Date")
idx_simed = idx_simed.drop("Adj Close", axis =1)

In [7]:
idx_simed

,Open,High,Low,Close,Volume
Date,,,,,
2020-04-20,10612.450195,10637.639648,10542.959961,10586.709961,2298700.0
2020-04-21,10544.799805,10544.799805,10278.950195,10288.419922,3392700.0
2020-04-22,10256.110352,10310.849609,10140.089844,10307.740234,2531000.0
2020-04-23,10370.129883,10462.700195,10294.879883,10366.509766,2633300.0
2020-04-24,10367.969727,10386.929688,10324.200195,10347.360352,2400100.0
...,...,...,...,...,...
2021-12-24,17966.349609,18039.849609,17953.570313,17961.640625,3834800.0
2021-12-25,17970.879883,18069.814453,17964.490235,18005.290039,3768000.0
2021-12-26,17973.145019,18084.796875,17969.950195,18027.114746,3734600.0


In [136]:
#train大model的資料 (length limit = 0) self label+dict label

classified_training_data = pd.read_pickle(f"D:\\論文相關\data\\classifier emotions\\classed_daily_emotion_categories_len_0_training_data.pkl")
dict_training_data = dict_emo_list[4]
dict_emo_list = dict_emo_list[:4]

columns_to_drop = classified_training_data.columns
classified_training_data["pos"] = dict_training_data["pos"] + classified_training_data["classified_pos"]
classified_training_data["neg"] = dict_training_data["neg"] + classified_training_data["classified_neg"]

classified_training_data["pos_ratio"] = classified_training_data["pos"]/(classified_training_data["pos"]+ classified_training_data["neg"])
classified_training_data["neg_ratio"] = classified_training_data["neg"]/(classified_training_data["pos"]+ classified_training_data["neg"])
classified_training_data = classified_training_data.drop(columns_to_drop, axis = 1)

for col in idx_simed:    
    classified_training_data[col] = idx_simed[col]
sorted_col = ["Open", "High", "Low", "Close", "Volume", "pos", "neg", "pos_ratio", "neg_ratio"]
classified_training_data = classified_training_data[sorted_col]
classified_training_data

,Open,High,Low,Close,Volume,pos,neg,pos_ratio,neg_ratio
2020-04-20,10612.450195,10637.639648,10542.959961,10586.709961,2298700.0,42,27,0.608696,0.391304
2020-04-21,10544.799805,10544.799805,10278.950195,10288.419922,3392700.0,168,91,0.648649,0.351351
2020-04-22,10256.110352,10310.849609,10140.089844,10307.740234,2531000.0,105,84,0.555556,0.444444
2020-04-23,10370.129883,10462.700195,10294.879883,10366.509766,2633300.0,191,64,0.749020,0.250980
2020-04-24,10367.969727,10386.929688,10324.200195,10347.360352,2400100.0,283,138,0.672209,0.327791
...,...,...,...,...,...,...,...,...,...
2021-12-24,17966.349609,18039.849609,17953.570313,17961.640625,3834800.0,1556,683,0.694953,0.305047
2021-12-25,17970.879883,18069.814453,17964.490235,18005.290039,3768000.0,1143,549,0.675532,0.324468
2021-12-26,17973.145019,18084.796875,17969.950195,18027.114746,3734600.0,818,294,0.735612,0.264388
2021-12-27,17975.410156,18099.779297,17975.410156,18048.939453,3701200.0,1167,654,0.640857,0.359143


In [142]:
dict_emo_list[3]

,pos,neg,pos_ratio,neg_ratio
2020-04-20,34,23,0.596491,0.403509
2020-04-21,138,73,0.654028,0.345972
2020-04-22,79,70,0.530201,0.469799
2020-04-23,138,46,0.750000,0.250000
2020-04-24,241,114,0.678873,0.321127
...,...,...,...,...
2021-12-24,1113,438,0.717602,0.282398
2021-12-25,933,392,0.704151,0.295849
2021-12-26,695,226,0.754615,0.245385
2021-12-27,865,465,0.650376,0.349624


In [143]:
#train大model的資料 (length limit = 0) dict label

for col in idx_simed:    
    dict_training_data[col] = idx_simed[col]
#dict_training_data = dict_training_data.drop("Adj Close", axis = 1)

dict_training_data = dict_training_data[sorted_col]
dict_training_data

,Open,High,Low,Close,Volume,pos,neg,pos_ratio,neg_ratio
2020-04-20,10612.450195,10637.639648,10542.959961,10586.709961,2298700.0,40,24,0.625000,0.375000
2020-04-21,10544.799805,10544.799805,10278.950195,10288.419922,3392700.0,147,76,0.659193,0.340807
2020-04-22,10256.110352,10310.849609,10140.089844,10307.740234,2531000.0,84,74,0.531646,0.468354
2020-04-23,10370.129883,10462.700195,10294.879883,10366.509766,2633300.0,180,47,0.792952,0.207048
2020-04-24,10367.969727,10386.929688,10324.200195,10347.360352,2400100.0,260,118,0.687831,0.312169
...,...,...,...,...,...,...,...,...,...
2021-12-24,17966.349609,18039.849609,17953.570313,17961.640625,3834800.0,1277,485,0.724745,0.275255
2021-12-25,17970.879883,18069.814453,17964.490235,18005.290039,3768000.0,1003,418,0.705841,0.294159
2021-12-26,17973.145019,18084.796875,17969.950195,18027.114746,3734600.0,723,234,0.755486,0.244514
2021-12-27,17975.410156,18099.779297,17975.410156,18048.939453,3701200.0,962,501,0.657553,0.342447


In [145]:
#製作不同字數(3-6)限制的情緒分數資料集 dict labeled 
#內為一個list 裝3-6字的 technical indicators and emotional indicators 
sim_stock_list_dict = []
for emo in dict_emo_list:
    temp_idx_simed = idx_simed[:]
    for col in emo.columns:
        cur_data = emo[col]
        temp_idx_simed[col] = cur_data
    #temp_idx_simed = temp_idx_simed.drop("Adj Close", axis =1)
    sim_stock_list_dict.append(temp_idx_simed)

c:\users\pppoe\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [146]:
sim_stock_list_dict[0]

,Open,High,Low,Close,Volume,pos,neg,pos_ratio,neg_ratio
Date,,,,,,,,,
2020-04-20,10612.450195,10637.639648,10542.959961,10586.709961,2298700.0,38,24,0.612903,0.387097
2020-04-21,10544.799805,10544.799805,10278.950195,10288.419922,3392700.0,144,75,0.657534,0.342466
2020-04-22,10256.110352,10310.849609,10140.089844,10307.740234,2531000.0,84,73,0.535032,0.464968
2020-04-23,10370.129883,10462.700195,10294.879883,10366.509766,2633300.0,170,47,0.783410,0.216590
2020-04-24,10367.969727,10386.929688,10324.200195,10347.360352,2400100.0,251,118,0.680217,0.319783
...,...,...,...,...,...,...,...,...,...
2021-12-24,17966.349609,18039.849609,17953.570313,17961.640625,3834800.0,1248,480,0.722222,0.277778
2021-12-25,17970.879883,18069.814453,17964.490235,18005.290039,3768000.0,990,413,0.705631,0.294369
2021-12-26,17973.145019,18084.796875,17969.950195,18027.114746,3734600.0,713,233,0.753700,0.246300


In [147]:
#製作不同字數(3-6)限制的情緒分數資料集 self labeled+dict labeled 
#內為一個list 裝3-6字的 technical indicators and self labeled+dict labeled 

sim_stock_list_classified_dict = []
for emo in dict_classified_emo_list:
    temp_idx_simed = idx_simed[:]
    for col in emo.columns:
        cur_data = emo[col]
        temp_idx_simed[col] = cur_data
        #temp_idx_simed = temp_idx_simed.drop("Adj Close", axis =1)
    sim_stock_list_classified_dict.append(temp_idx_simed)

c:\users\pppoe\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [148]:
sim_stock_list_classified_dict[0]

,Open,High,Low,Close,Volume,pos,neg,pos_ratio,neg_ratio
Date,,,,,,,,,
2020-04-20,10612.450195,10637.639648,10542.959961,10586.709961,2298700.0,40,27,0.597015,0.402985
2020-04-21,10544.799805,10544.799805,10278.950195,10288.419922,3392700.0,163,90,0.644269,0.355731
2020-04-22,10256.110352,10310.849609,10140.089844,10307.740234,2531000.0,104,83,0.556150,0.443850
2020-04-23,10370.129883,10462.700195,10294.879883,10366.509766,2633300.0,181,64,0.738776,0.261224
2020-04-24,10367.969727,10386.929688,10324.200195,10347.360352,2400100.0,273,138,0.664234,0.335766
...,...,...,...,...,...,...,...,...,...
2021-12-24,17966.349609,18039.849609,17953.570313,17961.640625,3834800.0,1509,671,0.692202,0.307798
2021-12-25,17970.879883,18069.814453,17964.490235,18005.290039,3768000.0,1120,538,0.675513,0.324487
2021-12-26,17973.145019,18084.796875,17969.950195,18027.114746,3734600.0,802,288,0.735780,0.264220


In [184]:
#把資料集切成需要的樣子 (feature(例如 high low, neg, pos...) 跟moving avg)
def data_prep(data_set,y_data, day_period = 3, emotion = True, moving_avg = True, 
              self_select_feature = False, pre_training_weight = (1,1)):
    

    #all_feature_name = list(data_set.drop("Close", axis = 1).columns) #原本所有的features
    all_feature_name = list(data_set.columns) #原本所有的features
    feature_name = list(data_set.columns) #最後要輸出的features
    if not type(self_select_feature) == list or not len(self_select_feature) :
        if not emotion:
            feature_name = list(data_set.columns[:5])
            
    else:
        feature_name = self_select_feature
    #print(feature_name)
    
    training_weight = {}
    for idx in range(len(feature_name)):
        if idx<4:
            training_weight[idx] = pre_training_weight[0] # 給technical indicators的權重 預設為1
        else:
            training_weight[idx] = pre_training_weight[1] # 給emotional indicators的權重 預設為1
    x = []
    y = []
    
    #print(feature_name)
    data_x = data_set[feature_name]
    

    #print(data_x)
    if moving_avg:
        temp_moving = data_x.rolling(window = day_period).mean().fillna(0)
        for i in range(0, len(temp_moving)-day_period):
            #第i天到第i+day_period天的X
            x.append(temp_moving.iloc[i:i+day_period].values)
            #i+day_period-1是上述天數的最後一天
            y.append(y_data[i+day_period])
            #index 是date
            #indexes.append(data_set.index[i+day_period-1])
    
    else:
        for i in range(0, len(data_set)-day_period):
            #第i天到第i+day_period天的X
            x.append(data_x.iloc[i:i+day_period].values)
            #i+day_period-1是上述天數的最後一天
            y.append(y_data[i+day_period])
            #index 是date
            #indexes.append(data_set.index[i+day_period-1])
        
    return np.array(x), np.array(y), feature_name , training_weight

In [185]:
#依照要跑的指標與日期 給出相對應的擬合模型
def experiment(day_period, data_set,text_len, file_path = "",retrain = True, mid_scaler = MinMaxScaler, y_label = "Close", predict_period = 60, weight = (1,1), emotion =False,  predict_day = 1 , FC_layer = 32, ver = 0, epoch = 100, loss = "mse", if_Conv = True):

    #data preparation
    if "Date" in data_set.columns:
        data_set = data_set.drop(["Date"], axis = 1)
        
    #正規化
    MMscaler_x_train = mid_scaler() 
    MMscaler_x_test = mid_scaler()
    MMscaler_y_train = mid_scaler()
    MMscaler_y_test = mid_scaler()
    
    #X資料集
    x = data_set.drop(y_label, axis = 1)
    #print(x.iloc[:-50, :])
    #X train
    x_train = x.iloc[:-predict_period, :]
    #print(x_train)
    #正規化後的X train
    x_train_scaled =  MMscaler_x_train.fit_transform(x_train)
    x_test = x.iloc[-predict_period:, :]
    #print(x_test)
    
    #正規化後的X test
    x_test_scaled =  MMscaler_x_test.fit_transform(x_test)
    
    #y dataset
    y = data_set[y_label]
    #print(y)
    y_train = np.array(y.iloc[:-predict_period]).reshape(-1, 1)
    
    #正規化後的Y train
    y_train_scaled =  MMscaler_y_train.fit_transform(y_train)
    
    #正規化後的Y test
    y_test = np.array(y.iloc[-predict_period:]).reshape(-1, 1)
    y_test_scaled =  MMscaler_y_test.fit_transform(y_test)
    
    if emotion: #因為情緒比例不能被正規化 所以要重新把比例賦值回去
        x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns)
        #print(x_train_scaled["pos_ratio"])
    
        x_train_scaled["pos_ratio"] = x_train["pos_ratio"].values
        x_train_scaled["neg_ratio"] = x_train["neg_ratio"].values
        #print(x_train_scaled["pos_ratio"])

        x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns)
        x_test_scaled["pos_ratio"] = x_test["pos_ratio"].values
        x_test_scaled["neg_ratio"] = x_test["neg_ratio"].values
        
    else: #如果沒有情緒的話就不用
        x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns)
        x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns)
    
    #print(x_test_scaled)
    

    #分成time steps的樣子
    training_data = data_prep(x_train_scaled, day_period = day_period, y_data = y_train_scaled, emotion= emotion, pre_training_weight = weight)
    training_data_x, training_data_y = training_data[0],training_data[1]
    training_weight= training_data[3]
    
    testing_data = data_prep(x_test_scaled, day_period = day_period, y_data = y_test_scaled, emotion= emotion, pre_training_weight = weight)
    testing_data_x, testing_data_y = testing_data[0],testing_data[1]
    
    #print(testing_data_x)
    
    #要不要用CNN 預設True
    if if_Conv:
        #加channel在最後面
        training_shape = (training_data_x.shape[1], training_data_x.shape[2], 1)
        
        training_data_x= np.reshape(training_data_x, (training_data_x.shape[0],training_data_x.shape[1],training_data_x.shape[2],1))
        testing_data_x = np.reshape(testing_data_x, (testing_data_x.shape[0],testing_data_x.shape[1],testing_data_x.shape[2],1))
    else:
        training_shape = training_data_x[0][0].shape
        
        
    #print(training_shape, training_data_x.shape, testing_data_x.shape)
    
    
    early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
    callbacks_list = [early_stop]
    #fit model
    if retrain:
        LSTM_model = model_dev(predict_day = predict_day,shape= training_shape, full_connect=FC_layer, loss_func = loss, if_Conv=if_Conv)
        #print(training_data_x[:5])
        history = LSTM_model.fit(x = training_data_x, y = training_data_y, class_weight = training_weight ,epochs=epoch, verbose = ver, validation_split=0.2, batch_size = 10, callbacks = callbacks_list)
        return MMscaler_y_test, MMscaler_y_test.inverse_transform(testing_data_y.reshape(-1, 1)), day_period, testing_data_x, LSTM_model
    
    
  
    return MMscaler_y_test, MMscaler_y_test.inverse_transform(testing_data_y.reshape(-1, 1)), day_period, testing_data_x
   

In [186]:
#建立模型

def model_dev(full_connect, predict_day = 1, shape = (5,6), loss_func = "mse", if_Conv = True):
    model = keras.models.Sequential()
    if if_Conv:
        model.add(TimeDistributed(Conv1D(100, 3, activation='relu'), input_shape=shape))
        model.add(TimeDistributed(MaxPooling1D(2, padding='same')))

        #model.add(BatchNormalization())

        # Flatten and dropout 
        model.add(TimeDistributed(Flatten()))
        model.add(Dropout(0.3))
        
        model.add(LSTM(512, return_sequences = True, activation='relu')) # , input_shape = shape
        model.add(Dropout(0.3))
        
    else:
        model.add(LSTM(512, return_sequences = True, input_shape = shape, activation='relu')) # 
        model.add(Dropout(0.6))
        #model.add(keras.layers.LSTM(256, return_sequences = True, activation = "relu"))
        #model.add(Dropout(0.5))
    model.add(LSTM(256, activation='relu'))
    model.add(Dropout(0.3))
    #全連接層
    model.add(Dense(full_connect, activation='relu'))
    #輸出
    model.add(Dense(predict_day))
    model.summary()
    
    opt = keras.optimizers.Adam(learning_rate=0.01)

    model.compile(optimizer=opt, loss=loss_func, metrics=[loss_func] ) #original :["accuracy",coeff_determination, r_square, ]
    
    return model


In [187]:
def get_result_list(stock_list, training_data, if_CNN_LSTM = True, if_emotion =True, weight =(1,1)):
    
    
    result_list =[]

    for data_period  in range(1,4):
        if if_CNN_LSTM:
            document_loc = "dict dom\\With_CNN\\LSTM_trained_model_weight_regression" #找出mse最小

        else:
            document_loc = "dict dom\\No_CNN\\LSTM_trained_model_weight_regression"


        cur_period = []
        print("before iter")
        length_counter = 3
        #用text len limits = 0的data(最上面的classified/dict_training_data) 去train的大model
        training_results = experiment(data_period, weight =weight, if_Conv= if_CNN_LSTM, emotion=if_emotion, data_set=training_data,epoch=100,  ver = 0, FC_layer=32, text_len = 3)
        
        #被text len = 0訓練好的model
        predict_model = training_results[-1] 
        
        
        for idx  in range(len(stock_list)): 
            #用不同text length的dataset去預測結果
            item = stock_list[idx]
            #print(item)
            doc_path  =(document_loc, data_period, length_counter)
            
            #利用上面min len = 0訓練好的 model來跑不同len limits的結果
            predict_result = experiment(data_period,retrain= False,  weight =weight, if_Conv= if_CNN_LSTM, file_path =  doc_path, emotion=if_emotion, data_set=item,epoch=100,  ver = 0, FC_layer=32, text_len = idx+3)
            print(f"predict with text length: {idx+3}")
            scaler = predict_result[0]
            y_test =predict_result[1]
            x_test = predict_result[3]
            
            #print(x_test)
            y_predict = scaler.inverse_transform(predict_model.predict(x_test).reshape(-1, 1))
            length_counter+=1
            cur_period.append((y_predict, y_test, predict_result[2], predict_result[3]))
            #print(y_predict)

        result_list.append(cur_period)
    return result_list


In [188]:
#self label+dict label跑出來的結果
result_list_dict_self_label = get_result_list(stock_list=sim_stock_list_classified_dict, training_data=classified_training_data, weight =(1,3))

before iter
Model: "sequential_98"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_294 (TimeDi (None, 1, 6, 100)         400       
_________________________________________________________________
time_distributed_295 (TimeDi (None, 1, 3, 100)         0         
_________________________________________________________________
time_distributed_296 (TimeDi (None, 1, 300)            0         
_________________________________________________________________
dropout_294 (Dropout)        (None, 1, 300)            0         
_________________________________________________________________
lstm_196 (LSTM)              (None, 1, 512)            1665024   
_________________________________________________________________
dropout_295 (Dropout)        (None, 1, 512)            0         
_________________________________________________________________
lstm_197 (LSTM)              (None, 256) 

In [189]:
metirc_self_label_dict = retrain_results(result_list_dict_self_label)
metirc_self_label_dict

,moving_avg_days 1,moving_avg_days 2,moving_avg_days 3
text_len 3,0.615385,0.625000,0.584615
text_len 4,0.593750,0.625000,0.584615
text_len 5,0.593750,0.603175,0.584615
text_len 6,0.593750,0.603175,0.584615


In [190]:
#dict label跑出來的結果
result_list_dict = get_result_list(stock_list=sim_stock_list_dict, training_data=dict_training_data, weight =(1,3))

before iter
Model: "sequential_101"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_303 (TimeDi (None, 1, 6, 100)         400       
_________________________________________________________________
time_distributed_304 (TimeDi (None, 1, 3, 100)         0         
_________________________________________________________________
time_distributed_305 (TimeDi (None, 1, 300)            0         
_________________________________________________________________
dropout_303 (Dropout)        (None, 1, 300)            0         
_________________________________________________________________
lstm_202 (LSTM)              (None, 1, 512)            1665024   
_________________________________________________________________
dropout_304 (Dropout)        (None, 1, 512)            0         
_________________________________________________________________
lstm_203 (LSTM)              (None, 256)

In [191]:
metric_dict= retrain_results(result_list_dict)
metric_dict

,moving_avg_days 1,moving_avg_days 2,moving_avg_days 3
text_len 3,0.666667,0.571429,0.575758
text_len 4,0.666667,0.571429,0.575758
text_len 5,0.666667,0.593750,0.575758
text_len 6,0.666667,0.593750,0.575758


In [192]:
#No emotion跑出來的結果
result_list_no_emo = get_result_list(stock_list=[idx_simed], training_data = idx_simed, if_emotion=False)


before iter
Model: "sequential_104"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_312 (TimeDi (None, 1, 2, 100)         400       
_________________________________________________________________
time_distributed_313 (TimeDi (None, 1, 1, 100)         0         
_________________________________________________________________
time_distributed_314 (TimeDi (None, 1, 100)            0         
_________________________________________________________________
dropout_312 (Dropout)        (None, 1, 100)            0         
_________________________________________________________________
lstm_208 (LSTM)              (None, 1, 512)            1255424   
_________________________________________________________________
dropout_313 (Dropout)        (None, 1, 512)            0         
_________________________________________________________________
lstm_209 (LSTM)              (None, 256)

In [193]:
no_emo_cur_results = retrain_results(result_list_no_emo)
no_emo_cur_results

,moving_avg_days 1,moving_avg_days 2,moving_avg_days 3
text_len 3,0.716418,0.666667,0.628571


# 存成持續性的檔案

In [1217]:
with open("D:\\論文相關\\static analysis\\20220405_result_list_self_label-5.pkl", "wb") as file:
    pickle.dump(result_list_dict_self_label, file)

In [1209]:
with open("D:\\論文相關\\static analysis\\202200405_result_list_dict-4.pkl", "wb") as file:
    pickle.dump(result_list_dict, file)

In [645]:
with open("D:\\論文相關\\static analysis\\20220327_result_list_no_emotion-2.pkl", "wb") as file:
    pickle.dump(result_list_no_emo, file)

In [26]:
with open("D:\\論文相關\\static analysis\\dict_emotion_predictions.pkl", "wb") as file:
    pickle.dump(result_list, file)

In [27]:
with open("D:\\論文相關\\static analysis\\No_emotion_predictions.pkl", "wb") as file:
    pickle.dump(result_list_no_emo, file)